In [1]:
# ✅ 超軽量SeqGAN（日本語版）完全版
# Google Colab対応

# ライブラリ
!pip install torch torchvision tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import random

# 🌱 データセット（日本語短文）
data = [
    "春の海ひねもすのたりのたりかな",
    "古池や蛙飛びこむ水の音",
    "柿食えば鐘が鳴るなり法隆寺",
    "山路来て何やらゆかしすみれ草",
    "夏草や兵どもが夢の跡",
    "秋深き隣は何をする人ぞ",
    "名月や池をめぐりて夜もすがら",
    "閑さや岩にしみ入る蝉の声",
    "五月雨を集めて早し最上川",
    "行く春や鳥啼き魚の目は泪"
]

chars = sorted(list(set("".join(data))))
char2idx = {c: i for i, c in enumerate(chars)}
idx2char = {i: c for c, i in char2idx.items()}
vocab_size = len(chars)

def encode(text): return [char2idx[c] for c in text]
def decode(indices): return "".join([idx2char[i] for i in indices])

encoded_data = [torch.tensor(encode(sentence)) for sentence in data]

# ⚙️ Generator
class Generator(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(Generator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x, hidden=None):
        x = self.embedding(x)
        out, hidden = self.lstm(x, hidden)
        logits = self.fc(out)
        return logits, hidden
    
    def sample(self, start_token, length):
        result = [start_token]
        input = torch.tensor([[start_token]])
        hidden = None
        for _ in range(length - 1):
            logits, hidden = self.forward(input, hidden)
            probs = torch.softmax(logits[:, -1, :], dim=-1)
            next_token = torch.multinomial(probs, 1).item()
            result.append(next_token)
            input = torch.tensor([[next_token]])
        return result

# ⚙️ Discriminator
class Discriminator(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_filters, filter_sizes):
        super(Discriminator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_filters, (fs, embed_dim)) for fs in filter_sizes
        ])
        self.fc = nn.Linear(num_filters * len(filter_sizes), 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)  # (batch, 1, seq_len, emb_dim)
        x = [torch.relu(conv(x)).squeeze(3) for conv in self.convs]
        x = [torch.max(t, dim=2)[0] for t in x]
        x = torch.cat(x, dim=1)
        return self.sigmoid(self.fc(x))

# 🌱 モデル初期化
generator = Generator(vocab_size, 16, 32)
discriminator = Discriminator(vocab_size, 16, 32, [2,3,4])

g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.01)
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
bce_loss = nn.BCELoss()

# 🔥 事前学習：Generator
print("📖 Generator Pre-Training")
for epoch in range(300):
    for sentence in encoded_data:
        inputs = sentence[:-1].unsqueeze(0)
        targets = sentence[1:].unsqueeze(0)
        logits, _ = generator(inputs)
        loss = criterion(logits.view(-1, vocab_size), targets.view(-1))
        g_optimizer.zero_grad()
        loss.backward()
        g_optimizer.step()
    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# 🔥 事前学習：Discriminator
print("\n📖 Discriminator Pre-Training")
real_label, fake_label = 1, 0
for epoch in range(100):
    for sentence in encoded_data:
        real_data = sentence.unsqueeze(0)
        fake_seq = generator.sample(random.choice(range(vocab_size)), len(sentence))
        fake_data = torch.tensor([fake_seq])
        
        real_out = discriminator(real_data)
        fake_out = discriminator(fake_data)
        
        real_loss = bce_loss(real_out, torch.ones_like(real_out)*real_label)
        fake_loss = bce_loss(fake_out, torch.ones_like(fake_out)*fake_label)
        d_loss = real_loss + fake_loss
        
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {d_loss.item():.4f}")

# 🔥 強化学習：Adversarial Training
print("\n🎯 Adversarial Training (RL)")
for epoch in range(100):
    for _ in range(len(data)):
        start_token = random.choice(range(vocab_size))
        fake_seq = generator.sample(start_token, 10)
        fake_data = torch.tensor([fake_seq])
        
        reward = discriminator(fake_data).detach()
        log_probs, _ = generator(fake_data[:,:-1])
        log_probs = torch.log_softmax(log_probs, dim=-1)
        selected_log_probs = log_probs.gather(2, fake_data[:,1:].unsqueeze(-1)).squeeze(-1)
        loss = -torch.mean(selected_log_probs * reward)
        
        g_optimizer.zero_grad()
        loss.backward()
        g_optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, G Loss: {loss.item():.4f}")

# 🌸 サンプル生成
print("\n🌸 生成サンプル")
for _ in range(5):
    start_token = random.choice(range(vocab_size))
    generated = generator.sample(start_token, 10)
    print("生成文:", decode(generated))



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\flare\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
📖 Generator Pre-Training
Epoch 0, Loss: 4.4581
Epoch 50, Loss: 0.0141
Epoch 100, Loss: 0.0042
Epoch 150, Loss: 0.0021
Epoch 200, Loss: 0.0012
Epoch 250, Loss: 0.0008

📖 Discriminator Pre-Training
Epoch 0, Loss: 2.7101
Epoch 20, Loss: 0.0808
Epoch 40, Loss: 2.8082
Epoch 60, Loss: 5.5304
Epoch 80, Loss: 0.0226

🎯 Adversarial Training (RL)
Epoch 0, G Loss: 0.1018
Epoch 10, G Loss: 0.0874
Epoch 20, G Loss: 0.1585
Epoch 30, G Loss: 0.1208
Epoch 40, G Loss: 0.0016
Epoch 50, G Loss: 0.0005
Epoch 60, G Loss: 0.0010
Epoch 70, G Loss: 0.0074
Epoch 80, G Loss: 0.0005
Epoch 90, G Loss: 0.0002

🌸 生成サンプル
生成文: く春や岩に岩に岩に岩
生成文: 集岩に岩に岩に岩に岩
生成文: 水さや岩に岩に岩に岩
生成文: 寺に食食食食食食食食
生成文: たさや岩に岩に岩に岩
